In [2]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pylab
import os
import cv2
from PIL import Image
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
# from https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb

In [3]:
coco_annotation = COCO("./merged_coco_annotation/merged_coco.json")

loading annotations into memory...
Done (t=0.31s)
creating index...
index created!


In [4]:
 # Category IDs.
cat_ids = coco_annotation.getCatIds()
print(f"Number of Unique Categories: {len(cat_ids)}")
print("Category IDs:")
print(cat_ids)  # The IDs are not necessarily consecutive.

Number of Unique Categories: 2
Category IDs:
[1, 2]


In [5]:
# All categories.
cats = coco_annotation.loadCats(cat_ids)
cat_names = [cat["name"] for cat in cats]
print("Categories Names:")
print(cat_names)

Categories Names:
['shark', 'human']


In [6]:
# Category ID -> Category Name.
query_id = cat_ids[0]
query_annotation = coco_annotation.loadCats([query_id])[0]
query_name = query_annotation["name"]
query_supercategory = query_annotation["supercategory"]
print("Category ID -> Category Name:")
print(
    f"Category ID: {query_id}, Category Name: {query_name}, Supercategory: {query_supercategory}"
)

Category ID -> Category Name:
Category ID: 1, Category Name: shark, Supercategory: object_label


In [7]:
# Category Name -> Category ID.
query_name = cat_names[0]
query_id = coco_annotation.getCatIds(catNms=[query_name])[0]
print("Category Name -> ID:")
print(f"Category Name: {query_name}, Category ID: {query_id}")

Category Name -> ID:
Category Name: shark, Category ID: 1


In [7]:
ann_ids = coco_annotation.getAnnIds()

In [8]:
print(len(ann_ids))

37169


In [9]:
# Get the ID of all the images containing the object of the category.
img_ids = coco_annotation.getImgIds(catIds=[query_id])
print(f"Number of Images Containing {query_name}: {len(img_ids)}")

Number of Images Containing shark: 7724


In [11]:
print(img_ids)

[100000, 100001, 100002, 100003, 100004, 100005, 100006, 100007, 100008, 100009, 100010, 100011, 100012, 100013, 100014, 100015, 100016, 100017, 100018, 100019, 100020, 100021, 100022, 100023, 100024, 100025, 100026, 100027, 100028, 100029, 100030, 100031, 100032, 100033, 100034, 100035, 100036, 100037, 100038, 100039, 100040, 100041, 100042, 100043, 100044, 100045, 100046, 100047, 100048, 100049, 100050, 100051, 100052, 100053, 100054, 100055, 100056, 100057, 100058, 100059, 100060, 100061, 100062, 100063, 100064, 100065, 100066, 100067, 100068, 100069, 100070, 100071, 100072, 100073, 100074, 100075, 100076, 100077, 100078, 100079, 100080, 100081, 100082, 100083, 100084, 100085, 100086, 100087, 100088, 100089, 100090, 100092, 100093, 100094, 100095, 100096, 100097, 100098, 100099, 100100, 100101, 100102, 100103, 100104, 100105, 100106, 100107, 100108, 100109, 100110, 100111, 100112, 100113, 100114, 100115, 100116, 100117, 100118, 100119, 100120, 100121, 100122, 100123, 100124, 100125,

In [12]:
# Pick one image.
img_id = img_ids[2]
img_info = coco_annotation.loadImgs([img_id])[0]
img_file_name = img_info["file_name"]
img_url = img_info["coco_url"]
print(
    f"Image ID: {img_id}, File Name: {img_file_name}, Image URL: {img_url}"
)

Image ID: 100002, File Name: 20200805_OneSharkSUPSurfers_00002.jpg, Image URL: 


In [13]:
# Get all the annotations for the specified image.
ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
anns = coco_annotation.loadAnns(ann_ids)
print(f"Annotations for Image ID {img_id}:")
print(anns[0])
print(anns[0]['image_id'])


Annotations for Image ID 100002:
{'id': 100002000, 'image_id': 100002, 'category_id': 1, 'segmentation': [[545.86, 1138.605, 609.488, 1138.605, 609.488, 1229.0230000000001, 545.86, 1229.0230000000001]], 'area': 5753.1165040000005, 'bbox': [545.86, 1138.605, 63.628, 90.418], 'iscrowd': 0}
100002


In [19]:
for i in range(1):
    # img_id = img_ids[i]
    img_id = 100091
    img_num = str(img_id - 100000)
    img_num = img_num.zfill(5)
    # Use URL to load image.
    im = Image.open(f"images/20200805_OneSharkSUPSurfers_{img_num}.jpg")
    
    ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
    anns = coco_annotation.loadAnns(ann_ids)

    width, height = (854, 480)
    my_dpi=96

    # print(im.size)

    fig, ax = plt.subplots(figsize=(width/my_dpi, height/my_dpi), dpi=my_dpi)
    ax.axis("off")
    ax.imshow(np.asarray(im))

    cat_id_to_text = {1: "shark", 2: "human"}
    
    # # Save image and its labeled version.
    # plt.axis("off")
    # plt.imshow(np.asarray(im))
    # plt.savefig(f"{img_id}.jpg", bbox_inches="tight", pad_inches=0)
    # Plot segmentation and bounding box.
    for ann in anns:
        box = ann['bbox']
        # cat_name = cat_id_to_text[ann['category_id']]
        bb = patches.Rectangle((box[0],box[1]), box[2],box[3], linewidth=1.5, edgecolor="red", facecolor="none")
        ax.add_patch(bb)
        ax.text(max(0, box[0] - 50), max(0, box[1] - 40), ann['category_id'], fontsize='small', color='white', bbox={'facecolor': 'red', 'alpha': 1, 'pad': 1})

    # coco_annotation.showAnns(anns, draw_bbox=True)

    # for i, ann in enumerate(anns):
    #     ax.text(anns[i]['bbox'][0], anns[i]['bbox'][1], anns[i]['category_id'], 
    #             bbox={'facecolor': 'None', 'alpha': 0.7, 'pad': 5})
    
    plt.savefig(f"check_ann_dont-save/{img_id}_annotated.png", bbox_inches="tight", pad_inches=0, dpi=my_dpi)

    plt.close()


In [32]:
def generate_video():
    image_folder = './check_ann_dont-save' # make sure to use your folder
    video_name = 'mygeneratedvideo.avi'
      
    images = [img for img in os.listdir(image_folder)
              if img.endswith(".jpg") or
                 img.endswith(".jpeg") or
                 img.endswith("png")]
     
    # Array images should only consider
    # the image files ignoring others if any
    print(images) 
  
    frame = cv2.imread(os.path.join(image_folder, images[0]))
  
  
    # setting the frame width, height width
    # the width, height of first image
    height, width, layers = frame.shape  
  
    # video = cv2.VideoWriter(video_name, 0, 1, (width, height)) 
  
    # # Appending the images to the video one by one
    # for image in images: 
    #     video.write(cv2.imread(os.path.join(image_folder, image))) 
      
    # # Deallocating memories taken for window creation
    # cv2.destroyAllWindows() 
    # video.release()  # releasing the video generated

In [33]:
generate_video()

['100017_annotated.png', '100000_annotated.png', '100004_annotated.png', '100013_annotated.png', '100010_annotated.png', '100023_annotated.png', '100009_annotated.png', '100020_annotated.png', '100025_annotated.png', '100021_annotated.png', '100018_annotated.png', '100008_annotated.png', '100022_annotated.png', '100006_annotated.png', '100011_annotated.png', '100015_annotated.png', '100002_annotated.png', '100005_annotated.png', '100001_annotated.png', '100016_annotated.png']


In [ ]:
coco_annotation.loadAnns(1)[0]["image_id"]

124

In [ ]:
sorted_image_list = [None for _ in range(len(img_ids))]

order = 0
curr_img_id = None

for ann_id in ann_ids:
    img_id = coco_annotation.loadAnns(ann_id)[0]["image_id"]
    
    if img_id != curr_img_id:
        # this is an image that has not been added to the list
        sorted_image_list[order] = img_id
        curr_img_id = img_id
        order += 1

print(sorted_image_list)

[124, 134, 144, 2, 4, 154, 5, 7, 9, 62, 10, 12, 14, 97, 15, 17, 19, 1, 3, 6, 8, 11, 13, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157, 158, 159, 160, 161, 162]


In [ ]:
str(123).zfill(5)

'00123'